In [1]:
from fairseq.models.roberta import RobertaModel
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
from fairseq.models.roberta import RobertaEncDecModel
import torch
import torch.nn.functional as F

/home/ubuntu/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-03 04:04:35 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX


In [2]:
path = "weights/nlp"
num_classes = 13
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

phoBERT_cls = RobertaModel.from_pretrained( path + '/PhoBERT_base_fairseq', checkpoint_file='model.pt')

# Load BPE
class BPE():
  bpe_codes = path + '/PhoBERT_base_fairseq/bpe.codes'

args = BPE()
phoBERT_cls.bpe = fastBPE(args) #Incorporate the BPE encoder into PhoBERT

# Add header cho classification với số lượng classes = 10
phoBERT_cls.register_classification_head('new_task', num_classes=num_classes)




2022-11-03 04:04:39 | INFO | fairseq.file_utils | loading archive file weights/nlp/PhoBERT_base_fairseq
2022-11-03 04:04:42 | INFO | fairseq.tasks.masked_lm | dictionary: 64000 types
2022-11-03 04:04:45 | INFO | fairseq.models.roberta.model | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 1, 'log_format': 'simple', 'log_file': None, 'aim_repo': None, 'aim_run_hash': None, 'tensorboard_logdir': '', 'wandb_project': None, 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': True, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_siz

In [ ]:
phoBERT_cls.load_state_dict(torch.load( path + "/phoBert_trained_state_dict/phoBert_state_dict_221101.pth"))
phoBERT_cls.to(device)
phoBERT_cls.eval()  # disable dropout (or leave in train mode to finetune

In [4]:
id2label = ['BHYT', 'address', 'admission_dates', 'age', 'department', 'diagnose',
 'discharge_dates', 'gender', 'hospital_name', 'note', 'patient_name',
 'sign_date', 'treatment']

In [7]:
tokens = 'TTYT Huyện Chư Sê'
token_idxs = phoBERT_cls.encode(tokens)
token_idxs.to(device)
logprobs = phoBERT_cls.predict('new_task', token_idxs)  # tensor([[-1.1050, -1.0672, -1.1245]], grad_fn=<LogSoftmaxBackward>)
output = F.softmax(logprobs, dim=-1)
cls = output.argmax()
print(output.max())
print(id2label[cls])

tensor(0.7205, device='cuda:0', grad_fn=<MaxBackward1>)
hospital_name


In [8]:
token_idxs

tensor([    0,  4577,  2142, 13906,  2454,     2])

In [1]:
import json

In [2]:
json_data = json.load(open("docbase_output.json"))

In [13]:
json_data["data"][0].keys()

dict_keys(['info', 'pages', 'type'])

In [25]:
json_data["data"][0]["info"]["pid"]

'038/120/'

In [21]:
json_data["data"][0]["info"].keys()

dict_keys(['address', 'address_box', 'address_confidence', 'department', 'department_box', 'department_confidence', 'diagnose', 'diagnose_box', 'diagnose_confidence', 'gender', 'gender_box', 'gender_confidence', 'hospital_discharge_date', 'hospital_discharge_date_box', 'hospital_discharge_date_confidence', 'hospitalization_date', 'hospitalization_date_box', 'hospitalization_date_confidence', 'icd_10', 'image', 'image_seals', 'medical_facility', 'medical_facility_box', 'medical_facility_confidence', 'patient_name', 'patient_name_box', 'patient_name_confidence', 'pid', 'pid_box', 'pid_confidence', 'treatments', 'treatments_box', 'treatments_confidence', 'valid_seals', 'year_of_birth', 'year_of_birth_box', 'year_of_birth_confidence'])

In [29]:
json_data["data"][0]["info"].keys()

dict_keys(['address', 'address_box', 'address_confidence', 'department', 'department_box', 'department_confidence', 'diagnose', 'diagnose_box', 'diagnose_confidence', 'gender', 'gender_box', 'gender_confidence', 'hospital_discharge_date', 'hospital_discharge_date_box', 'hospital_discharge_date_confidence', 'hospitalization_date', 'hospitalization_date_box', 'hospitalization_date_confidence', 'icd_10', 'image', 'image_seals', 'medical_facility', 'medical_facility_box', 'medical_facility_confidence', 'patient_name', 'patient_name_box', 'patient_name_confidence', 'pid', 'pid_box', 'pid_confidence', 'treatments', 'treatments_box', 'treatments_confidence', 'valid_seals', 'year_of_birth', 'year_of_birth_box', 'year_of_birth_confidence'])

In [27]:
empty_result = {
    "data": [
        {
        "info":
            {
                "gender": None,
                "gender_box": [],
                "gender_confidence": 0,
                
                "hospital_discharge_date": None,
                "hospital_discharge_date_box": [],
                "hospital_discharge_date_confidence": 0,
                
                "hospitalization_date": None,
                "hospitalization_date_box": [],
                "hospitalization_date_confidence": 0,
                
                "icd_10": [],
                
                "medical_facility": None,
                "medical_facility_box": [],
                "medical_facility_confidence": 0,
                
                "patient_name": None,
                "patient_name_box": [],
                "patient_name_confidence": 0,
                
                "year_of_birth": None,
                "year_of_birth_box": [],
                "year_of_birth_confidence": 0,
                
                "image_seals": "UN_CHECKED",

            },
        "pages": [],
        "type": None,
        },
    ]
}

In [28]:
empty_result

{'data': [{'info': {'gender': None,
    'gender_box': [],
    'gender_confidence': 0,
    'hospital_discharge_date': None,
    'hospital_discharge_date_box': [],
    'hospital_discharge_date_confidence': 0,
    'hospitalization_date': None,
    'hospitalization_date_box': [],
    'hospitalization_date_confidence': 0,
    'icd_10': {},
    'medical_facility': None,
    'medical_facility_box': [],
    'medical_facility_confidence': 0,
    'patient_name': None,
    'patient_name_box': [],
    'patient_name_confidence': 0,
    'year_of_birth': None,
    'year_of_birth_box': [],
    'year_of_birth_confidence': 0,
    'date_of_birth': None,
    'date_of_birth_box': [],
    'date_of_birth_confidence': 0,
    'age': None,
    'age_box': [],
    'age_confidence': 0,
    'image_seals': 'NOTCHECK'},
   'pages': [],
   'type': None}]}

In [3]:
json_data

{'data': [{'info': {'address': 'Xã Thọ Lâm Huyện Thọ Xuân Thanh Hóa',
    'address_box': [262, 328, 702, 367],
    'address_confidence': 0.9982713460922241,
    'department': 'Khoa Truyền nhiễm',
    'department_box': [106, 134, 309, 173],
    'department_confidence': 0.997370719909668,
    'diagnose': 'J20.9 Viêm phế quản cấp, không phân loại/ J02.9 Viêm họng cấp. không đặc hiệu',
    'diagnose_box': [298, 428, 1203, 476],
    'diagnose_confidence': 0.5862256288528442,
    'gender': 'Nữ',
    'gender_box': [1130, 228, 1172, 266],
    'gender_confidence': 0.999955415725708,
    'hospital_discharge_date': '15/07/2022',
    'hospital_discharge_date_box': [461, 394, 756, 434],
    'hospital_discharge_date_confidence': 0.9998311996459961,
    'hospitalization_date': '12/07/2022',
    'hospitalization_date_box': [465, 362, 765, 398],
    'hospitalization_date_confidence': 0.9998764991760254,
    'icd_10': [['J02.9', [779, 431, 840, 468], 0.9995610117912292],
     ['J20.9', [298, 428, 360, 4